# 05 Tooling | 01 Semantic Kernel | 03 Tools 

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment. 

## Step 1: Create Semantic Kernel

[MS Semantic Kernel](https://github.com/microsoft/semantic-kernel) is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.

A instance of `Kernel` is created and acts as centralized object to execute Semantic Kernel functionality.


In [7]:
#r "nuget: Microsoft.SemanticKernel, 1.15.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.15.1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using DotNetEnv;

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";
string assetFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";  


var kernel = Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        chatCompletionDeploymentName,
        oAiEndpoint,
        oAiApiKey
    )
    .Build();
    
Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.15.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.15.1

Semantic Kernel created...


## Step 3: Add Plugins to the planner

The two native functions: 

- `GetSupportEventWinner()`
- `GetSportEventResult()`

are defined as `KernelFunction` and provided to the `kernel` instance created in the previous code cell.


In [8]:
private string GetSportEventWinner(string sportEvent, string year)
{
    // Implement the logic to get the winner of the sport event.
    Console.WriteLine("GetSportEventWinner called...");
    return "Kansas City Chiefs";
}

private string GetSportEventResult(string sportEvent, string year)
{
    // Implement the logic to get the result of the sport event.
    Console.WriteLine("GetSportEventResult called...");
    return "25:22";
}

KernelFunction kernelFunctionWinner = kernel.CreateFunctionFromMethod(
    GetSportEventWinner, 
    "GetSportEventWinner", 
    "Get the winner of the sport event."
);

KernelFunction kernelFunctionResult = kernel.CreateFunctionFromMethod(
    GetSportEventResult, 
    "GetSportEventResult", 
    "Get the result of the sport event."
);

kernel.ImportPluginFromFunctions(
    "SportFunctions", 
    new KernelFunction[] { 
        kernelFunctionWinner, 
        kernelFunctionResult 
    }
);

Console.WriteLine($"Sport functions imported...");

Sport functions imported...


## Step 4: Call Functions

The `ChatCompletionService` instance is requested from the previously defined `kernel` and used to call the `GetChatMessageContentAsync()` function. Due to the configuration in `OpenAIPromptExecutionSessings` which allows to `AutoInvokeKernelFunctions` the provided native functions are called by the Semantic Kernel functionality and on request from the LLM.

In [10]:
IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>(); 

// Allow Auto Invoke of provided plugins
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

ChatHistory chatHistory = new ChatHistory();
chatHistory.AddUserMessage("Who won the Super Bowl 2024? What was the result?");

ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
    chatHistory,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

Console.WriteLine($"Execution results: {chatMessageContent}");
Console.WriteLine($"See the wrong score as proof that local functions have been called!")

GetSportEventWinner called...
GetSportEventResult called...
Execution results: The winner of the Super Bowl 2024 was the Kansas City Chiefs. The result of the game was 25-22.
See the wrong score as proof that local functions have been called!
